# Olog Module for Jupyter with SQLite Integration

This notebook defines classes for representing Ologs (Ontology Logs)
and integrates basic SQLite database interactions for storing instances
when the conceptual target category is `Set`.

**Features:**
- Olog structure representation (Objects, Morphisms).
- Optional validation for object labels ("a"/"an").
- SQLite database association.
- Automatic schema generation (Tables for Objects, Columns/FKs for functional Morphisms).
- Methods for adding and querying instances via the database.
- Graphviz visualization.

**Dependencies:**
- `graphviz` Python library (`pip install graphviz`) and system installation.
- `sqlite3` (standard library).

In [ ]:
# Necessary Imports
import warnings
import sqlite3
import re
import os
from typing import Optional, Dict, List, Any, Set, Tuple, Union

# Optional, but needed for visualization rendering in Jupyter
try:
    import graphviz
    GRAPHVIZ_AVAILABLE = True
except ImportError:
    GRAPHVIZ_AVAILABLE = False
    warnings.warn("Graphviz library not found. Visualization methods will not render inline.")

In [ ]:
# Helper function to sanitize labels for SQL table/column names
def sanitize_for_sql(label: str) -> str:
    # Remove leading 'a ' or 'an ' if present
    label = re.sub(r'^(a|an)\s+', '', label, flags=re.IGNORECASE)
    # Replace spaces and invalid characters with underscores
    label = re.sub(r'\W|^(?=\d)', '_', label)
    # Remove consecutive underscores
    label = re.sub(r'_+', '_', label)
    # Remove leading/trailing underscores
    label = label.strip('_')
    return label.lower() or "unnamed" # Ensure not empty

In [ ]:
class OlogObject:
    """Represents an object (box) in an Olog."""
    def __init__(self, label: str, description: Optional[str] = None):
        if not isinstance(label, str) or not label:
            raise ValueError("OlogObject label must be a non-empty string.")
        self.label: str = label
        self.description: Optional[str] = description
        # Derived table name (lazy initialized by Olog)
        self._table_name: Optional[str] = None

    @property
    def table_name(self) -> str:
        if self._table_name is None:
             # Should be set by Olog upon adding the object
             raise AttributeError("Table name not initialized for OlogObject.")
        return self._table_name

    def __str__(self) -> str:
        return self.label

    def __repr__(self) -> str:
        return f'OlogObject(label="{self.label}")'

    def __eq__(self, other) -> bool:
        if isinstance(other, OlogObject):
            return self.label == other.label
        return False

    def __hash__(self) -> int:
        return hash(self.label)

In [ ]:
class OlogMorphism:
    """Represents a morphism (arrow) in an Olog."""
    def __init__(self,
                 source: OlogObject,
                 target: OlogObject,
                 label: str,
                 description: Optional[str] = None,
                 is_functional_fk: bool = False):
        """
        Initializes an OlogMorphism.

        Args:
            source: The source OlogObject (domain).
            target: The target OlogObject (codomain).
            label: The verb phrase defining the relationship (e.g., "has").
            description: An optional longer description.
            is_functional_fk: If True, indicates this morphism represents a
                              functional dependency (N-to-1 or 1-to-1) to be
                              represented as a foreign key column in the source
                              object's table. Defaults to False.
        """
        if not isinstance(source, OlogObject) or not isinstance(target, OlogObject):
            raise TypeError("Morphism source and target must be OlogObject instances.")
        if not isinstance(label, str) or not label:
            raise ValueError("OlogMorphism label must be a non-empty string.")

        self.source: OlogObject = source
        self.target: OlogObject = target
        self.label: str = label
        self.description: Optional[str] = description
        self.is_functional_fk: bool = is_functional_fk
        # Derived column name (lazy initialized by Olog)
        self._column_name: Optional[str] = None

    @property
    def column_name(self) -> Optional[str]:
        if not self.is_functional_fk:
            return None
        if self._column_name is None:
            # Should be set by Olog upon adding the morphism
            raise AttributeError("Column name not initialized for functional OlogMorphism.")
        return self._column_name

    def __str__(self) -> str:
        fk_marker = " (FK)" if self.is_functional_fk else ""
        return f'"{self.source.label}" --[{self.label}{fk_marker}]--> "{self.target.label}"'

    def __repr__(self) -> str:
        return (f'OlogMorphism(source="{self.source.label}", '
                f'target="{self.target.label}", label="{self.label}", '
                f'is_functional_fk={self.is_functional_fk})')

    def __eq__(self, other) -> bool:
        if isinstance(other, OlogMorphism):
            return (self.source == other.source and
                    self.target == other.target and
                    self.label == other.label and
                    self.is_functional_fk == other.is_functional_fk) # Include flag in equality
        return False

    def __hash__(self) -> int:
        return hash((self.source, self.target, self.label, self.is_functional_fk))

In [ ]:
class Olog:
    """Represents an Ontology Log (Olog) with SQLite integration."""
    def __init__(self,
                 name: str,
                 db_path: Optional[str] = None, # Path to SQLite DB file
                 description: Optional[str] = None,
                 target_category: str = "Set",
                 validate_labels: bool = False):
        """
        Initializes an Olog, optionally associated with an SQLite database.

        Args:
            name: The name identifying this olog or domain.
            db_path: Path to the SQLite database file. If None, DB features disabled.
            description: An optional description of the olog's domain/purpose.
            target_category: Descriptive target category (e.g., "Set").
            validate_labels: If True, warn about object labels not starting with "a/an".
        """
        self.name: str = name
        self.description: Optional[str] = description
        self.target_category: str = target_category
        self.validate_labels: bool = validate_labels

        self._objects: Dict[str, OlogObject] = {}
        self._morphisms: Set[OlogMorphism] = set()
        self._morphisms_from: Dict[str, List[OlogMorphism]] = {}
        self._morphisms_to: Dict[str, List[OlogMorphism]] = {}

        # --- Database Attributes ---
        self.db_path: Optional[str] = db_path
        self._conn: Optional[sqlite3.Connection] = None
        self._active_cursors: List[sqlite3.Cursor] = [] # Track open cursors


    # --- Olog Structure Methods ---
    def add_object(self, label: str, description: Optional[str] = None) -> OlogObject:
        """Adds a new object (type/box) to the olog."""
        if label in self._objects:
            raise ValueError(f"Object with label '{label}' already exists in olog '{self.name}'.")

        if self.validate_labels:
            if not (label.startswith('a ') or label.startswith('an ')):
                warnings.warn(
                    f"Object label '{label}' does not start with 'a' or 'an'. "
                    f"(Validation enabled for olog '{self.name}')"
                )

        obj = OlogObject(label, description)
        # Eagerly assign sanitized table name
        obj._table_name = sanitize_for_sql(label)
        if not obj._table_name:
             warnings.warn(f"Could not generate valid table name for object: '{label}'")

        self._objects[label] = obj
        self._morphisms_from[label] = []
        self._morphisms_to[label] = []
        return obj

    def get_object(self, label: str) -> Optional[OlogObject]:
        """Retrieves an object by its label."""
        return self._objects.get(label)

    def add_morphism(self,
                     source_label: str,
                     target_label: str,
                     label: str,
                     description: Optional[str] = None,
                     is_functional_fk: bool = False) -> OlogMorphism:
        """Adds a new morphism (relationship/arrow) to the olog."""
        source_obj = self.get_object(source_label)
        if source_obj is None:
            raise ValueError(f"Source object '{source_label}' not found in olog '{self.name}'.")

        target_obj = self.get_object(target_label)
        if target_obj is None:
            raise ValueError(f"Target object '{target_label}' not found in olog '{self.name}'.")

        morphism = OlogMorphism(source_obj, target_obj, label, description, is_functional_fk)

        if morphism in self._morphisms:
             raise ValueError(f"Morphism '{morphism}' already exists in olog '{self.name}'.")

        # Assign column name if it's a functional FK
        if morphism.is_functional_fk:
             col_name_base = sanitize_for_sql(morphism.label)
             # Append target table name for clarity, avoiding collisions
             # col_name = f"{col_name_base}_to_{morphism.target.table_name}_id"
             col_name = f"{col_name_base}_id" # Simpler, assumes label is descriptive enough
             morphism._column_name = col_name

        self._morphisms.add(morphism)
        self._morphisms_from[source_label].append(morphism)
        self._morphisms_to[target_label].append(morphism)
        return morphism

    @property
    def objects(self) -> List[OlogObject]:
        return list(self._objects.values())

    @property
    def morphisms(self) -> List[OlogMorphism]:
        return list(self._morphisms)

    def get_morphisms_from(self, source_label: str) -> List[OlogMorphism]:
        if source_label not in self._objects:
             raise ValueError(f"Source object '{source_label}' not found.")
        return list(self._morphisms_from.get(source_label, []))

    def get_morphisms_to(self, target_label: str) -> List[OlogMorphism]:
        if target_label not in self._objects:
             raise ValueError(f"Target object '{target_label}' not found.")
        return list(self._morphisms_to.get(target_label, []))

    # --- Database Connection Methods ---
    def connect(self) -> sqlite3.Connection:
        """Establishes connection to the SQLite database. Returns the connection."""
        if not self.db_path:
            raise ConnectionError("Database path (db_path) is not set for this Olog.")
        if self._conn is None:
            try:
                # Ensure directory exists
                db_dir = os.path.dirname(self.db_path)
                if db_dir and not os.path.exists(db_dir):
                    os.makedirs(db_dir)
                self._conn = sqlite3.connect(self.db_path)
                self._conn.row_factory = sqlite3.Row # Access columns by name
                # Enable foreign key support
                with self._conn:
                    self._conn.execute("PRAGMA foreign_keys = ON;")
            except sqlite3.Error as e:
                raise ConnectionError(f"Failed to connect to database at '{self.db_path}': {e}")
        return self._conn

    def close(self):
        """Closes the database connection if open."""
        # Close any remaining cursors first (important!)
        for cursor in self._active_cursors:
            try:
                cursor.close()
            except sqlite3.Error:
                pass # Ignore errors if cursor is already invalid
        self._active_cursors = []

        if self._conn is not None:
            try:
                self._conn.close()
            except sqlite3.Error as e:
                 warnings.warn(f"Error closing database connection: {e}")
            finally:
                self._conn = None

    def get_cursor(self) -> sqlite3.Cursor:
        """Gets a cursor for the current database connection."""
        conn = self.connect() # Ensure connection exists
        cursor = conn.cursor()
        self._active_cursors.append(cursor) # Track cursor
        return cursor

    def _release_cursor(self, cursor: sqlite3.Cursor):
        """Releases a tracked cursor."""
        try:
            cursor.close()
        except sqlite3.Error:
            pass # Ignore if already closed
        if cursor in self._active_cursors:
             self._active_cursors.remove(cursor)

    def __enter__(self):
        """Context manager entry: connect."""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit: close connection."""
        self.close()


    # --- Schema and Instance Methods ---

    def create_tables_from_olog(self, drop_existing=False):
        """
        Generates and executes SQL CREATE TABLE statements based on the Olog structure.

        Args:
            drop_existing: If True, drops tables before creating them. USE WITH CAUTION!
        """
        if not self.db_path:
            warnings.warn("No db_path set. Cannot create database tables.")
            return

        cursor = self.get_cursor()
        try:
            # Get existing tables to handle dropping correctly
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            existing_tables = {row[0] for row in cursor.fetchall()}

            create_statements = []
            table_names = set()

            # --- Generate CREATE TABLE statements ---
            for obj in self.objects:
                if not obj.table_name:
                    warnings.warn(f"Skipping table creation for object without table name: {obj.label}")
                    continue

                table_name = obj.table_name
                if table_name in table_names:
                     warnings.warn(f"Duplicate table name '{table_name}' detected. Skipping regeneration for: {obj.label}")
                     continue
                table_names.add(table_name)

                if drop_existing and table_name in existing_tables:
                    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
                    print(f"Dropped table {table_name}")

                columns = [
                    "id INTEGER PRIMARY KEY AUTOINCREMENT",
                    "value TEXT" # Basic value column
                ]
                foreign_keys = []

                # Add columns for functional morphisms originating from this object
                for morph in self.get_morphisms_from(obj.label):
                    if morph.is_functional_fk and morph.column_name:
                        target_table = morph.target.table_name
                        fk_col_name = morph.column_name
                        columns.append(f"{fk_col_name} INTEGER")
                        foreign_keys.append(
                            f"FOREIGN KEY ({fk_col_name}) REFERENCES {target_table}(id)"
                        )

                columns_sql = ",\n    ".join(columns + foreign_keys)
                sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n    {columns_sql}\n);"
                create_statements.append(sql)

            # --- Execute statements ---
            conn = self._conn # Use the existing connection
            with conn: # Use transaction
                for sql in create_statements:
                     # print(f"Executing: {sql}") # Debugging
                     cursor.execute(sql)
            print(f"Schema generation complete for olog '{self.name}'.")

        except sqlite3.Error as e:
            print(f"Database error during schema creation: {e}")
            # Consider rolling back if needed, though auto-commit handles some cases
        finally:
            self._release_cursor(cursor)


    def add_instance(self, obj_label: str, value: Any, **relationships: Dict[str, int]) -> Optional[int]:
        """
        Adds an instance to the table corresponding to the OlogObject.

        Args:
            obj_label: The label of the OlogObject (type) to add an instance for.
            value: The primary value for the instance (will be stored in 'value' column).
            relationships: Keyword arguments where keys are *morphism labels*
                           (representing functional FKs) and values are the IDs
                           of the related instances in the target tables.
                           e.g., has_side_chain=3

        Returns:
            The ID of the newly inserted instance, or None if insertion failed.
        """
        obj = self.get_object(obj_label)
        if obj is None or not obj.table_name:
            raise ValueError(f"Object '{obj_label}' not found or has no associated table name.")

        cursor = self.get_cursor()
        try:
            cols_to_insert = ["value"]
            vals_to_insert = [str(value)] # Store value as TEXT for simplicity
            placeholders = ["?"]

            # Map relationship kwargs to column names and values
            valid_fk_morphs = {
                m.label: m for m in self.get_morphisms_from(obj_label)
                if m.is_functional_fk and m.column_name
            }

            for morph_label, target_id in relationships.items():
                if morph_label in valid_fk_morphs:
                    morphism = valid_fk_morphs[morph_label]
                    cols_to_insert.append(morphism.column_name)
                    vals_to_insert.append(target_id)
                    placeholders.append("?")
                else:
                    warnings.warn(f"Morphism label '{morph_label}' provided in relationships "
                                  f"is not a recognized functional FK for object '{obj_label}'. Skipping.")

            cols_sql = ", ".join(cols_to_insert)
            placeholders_sql = ", ".join(placeholders)
            sql = f"INSERT INTO {obj.table_name} ({cols_sql}) VALUES ({placeholders_sql});"

            # print(f"Executing: {sql} with {vals_to_insert}") # Debugging
            conn = self._conn
            with conn:
                cursor.execute(sql, vals_to_insert)
            return cursor.lastrowid # Get the ID of the inserted row

        except sqlite3.Error as e:
            print(f"Database error adding instance to '{obj.table_name}': {e}")
            return None
        finally:
            self._release_cursor(cursor)


    def get_instances(self, obj_label: str) -> List[sqlite3.Row]:
        """Retrieves all instances for a given OlogObject label."""
        obj = self.get_object(obj_label)
        if obj is None or not obj.table_name:
            raise ValueError(f"Object '{obj_label}' not found or has no associated table name.")

        cursor = self.get_cursor()
        try:
            sql = f"SELECT * FROM {obj.table_name};"
            cursor.execute(sql)
            return cursor.fetchall()
        except sqlite3.Error as e:
            print(f"Database error getting instances from '{obj.table_name}': {e}")
            return []
        finally:
            self._release_cursor(cursor) # Ensure cursor is closed

    def get_instance_by_id(self, obj_label: str, instance_id: int) -> Optional[sqlite3.Row]:
        """Retrieves a specific instance by its ID."""
        obj = self.get_object(obj_label)
        if obj is None or not obj.table_name:
            raise ValueError(f"Object '{obj_label}' not found or has no associated table name.")

        cursor = self.get_cursor()
        try:
            sql = f"SELECT * FROM {obj.table_name} WHERE id = ?;"
            cursor.execute(sql, (instance_id,))
            return cursor.fetchone()
        except sqlite3.Error as e:
            print(f"Database error getting instance ID {instance_id} from '{obj.table_name}': {e}")
            return None
        finally:
            self._release_cursor(cursor)

    def get_related_instance(self, source_obj_label: str, source_instance_id: int, morphism_label: str) -> Optional[sqlite3.Row]:
        """
        Gets the related instance following a functional FK morphism.

        Args:
            source_obj_label: Label of the source object.
            source_instance_id: ID of the source instance row.
            morphism_label: The label of the functional FK morphism to follow.

        Returns:
            The related instance row from the target table, or None if not found/error.
        """
        source_obj = self.get_object(source_obj_label)
        if source_obj is None:
             raise ValueError(f"Source object '{source_obj_label}' not found.")

        # Find the relevant morphism
        morphism = None
        for m in self.get_morphisms_from(source_obj_label):
            if m.label == morphism_label and m.is_functional_fk and m.column_name:
                morphism = m
                break

        if morphism is None:
            raise ValueError(f"Functional FK morphism '{morphism_label}' not found originating from '{source_obj_label}'.")

        # Get the source instance to find the foreign key value
        source_instance = self.get_instance_by_id(source_obj_label, source_instance_id)
        if source_instance is None:
            print(f"Source instance ID {source_instance_id} not found in table for '{source_obj_label}'.")
            return None

        fk_column_name = morphism.column_name
        target_id = source_instance[fk_column_name] # Access FK value by column name

        if target_id is None:
            # Relationship might be optional/not set
            return None

        # Get the target instance using the found ID
        target_obj_label = morphism.target.label
        return self.get_instance_by_id(target_obj_label, target_id)


    # --- Visualization ---
    def to_dot(self) -> str:
        """Generates a DOT language string for visualizing the olog with Graphviz."""
        dot_lines = [f'digraph "{self.name}" {{']
        dot_lines.append('  rankdir=LR; # Left-to-right layout')
        dot_lines.append('  node [shape=box, style=rounded];')

        for obj_label in sorted(self._objects.keys()):
             dot_lines.append(f'  "{obj_label}" [label="{obj_label}"];')

        sorted_morphisms = sorted(
            list(self._morphisms),
            key=lambda m: (m.source.label, m.target.label, m.label)
        )
        for morph in sorted_morphisms:
             # Add marker for functional FK morphisms in label
             label = morph.label
             if morph.is_functional_fk:
                 label += " (FK)"
             dot_lines.append(f'  "{morph.source.label}" -> "{morph.target.label}" [label="{label}"];')

        dot_lines.append('}')
        return "\n".join(dot_lines)

    def view(self):
        """Renders the olog visualization directly in Jupyter (if graphviz is available)."""
        if not GRAPHVIZ_AVAILABLE:
            print("Graphviz is not installed or importable. Cannot render visualization.")
            print("DOT representation:\n", self.to_dot())
            return None
        try:
            return graphviz.Source(self.to_dot())
        except Exception as e:
            print(f"Error during Graphviz rendering: {e}")
            print("DOT representation:\n", self.to_dot())
            return None

    def __str__(self) -> str:
        # (Keep the __str__ method from the previous version, it's good)
        s = f"Olog(Name: '{self.name}'"
        if self.description: s += f", Description: '{self.description}'"
        s += f", Target Category: '{self.target_category}'"
        s += f", Validate Labels: {self.validate_labels}"
        s += f", DB Path: '{self.db_path or 'None'}'"
        s += ")\n"
        s += "Objects:\n"
        if not self._objects: s += "  (None)\n"
        else:
            for obj_label in sorted(self._objects.keys()):
                 s += f"  - {self._objects[obj_label].label} (Table: {self._objects[obj_label].table_name or 'N/A'})\n"
        s += "Morphisms:\n"
        if not self._morphisms: s += "  (None)\n"
        else:
            sorted_morphisms = sorted(list(self._morphisms), key=lambda m: (m.source.label, m.target.label, m.label))
            for morph in sorted_morphisms: s += f"  - {morph}\n" # Morph's __str__ includes FK marker
        return s

---
## Example Usage with SQLite Integration

1. Define the Olog structure.
2. Associate it with a database file.
3. Create the database schema (tables).
4. Add instances (data).
5. Query instances and follow relationships.

In [ ]:
# --- 1. Define Olog Structure ---
print("--- Defining Olog Structure ---")
db_file = "amino_acids.db"
# Remove existing DB file for a clean run (optional)
if os.path.exists(db_file):
    os.remove(db_file)

# Create Olog with DB path
amino_acid_olog = Olog(
    name="Amino Acid Structure DB",
    db_path=db_file,
    description="Simplified view of amino acid components with DB persistence",
    validate_labels=False # Disabled validation for brevity
)

# Add objects
amino_acid_olog.add_object("an amino acid")
amino_acid_olog.add_object("a side chain")
amino_acid_olog.add_object("an alpha carbon")
# amino_acid_olog.add_object("a carboxyl group") # Let's omit these for a simpler FK example
# amino_acid_olog.add_object("an amino group")

# Add morphisms - Mark 'has' as functional FK
# (Each amino acid has exactly one side chain and one alpha carbon in this simple model)
amino_acid_olog.add_morphism("an amino acid", "a side chain", "has_side_chain", is_functional_fk=True)
amino_acid_olog.add_morphism("an amino acid", "an alpha carbon", "has_alpha_carbon", is_functional_fk=True)
amino_acid_olog.add_morphism("an alpha carbon", "a side chain", "is_bonded_to_side_chain") # Not an FK from alpha C here


print(amino_acid_olog)

---
## 2. Visualize the Structure (includes FK markers)

In [ ]:
# Visualize the structure
viz = amino_acid_olog.view()
viz

---
## 3. Create Database Schema

This connects to the database and executes `CREATE TABLE` statements.

In [ ]:
# --- 3. Create Schema ---
print("\n--- Creating Database Schema ---")
# Use context manager for connection handling
try:
    with amino_acid_olog as olog_db:
        olog_db.create_tables_from_olog(drop_existing=True) # Drop tables if they exist
except (ConnectionError, sqlite3.Error) as e:
    print(f"Database operation failed: {e}")

---
## 4. Add Instances (Data)

We add instances to the tables. Note how we need the IDs of related instances
(side chains, alpha carbons) when adding the main amino acid instances.

In [ ]:
# --- 4. Add Instances ---
print("\n--- Adding Instances ---")
try:
    with amino_acid_olog as olog_db:
        # Add some side chains first, get their IDs
        sc_h_id = olog_db.add_instance("a side chain", "H")       # Glycine side chain
        sc_ch3_id = olog_db.add_instance("a side chain", "CH3")    # Alanine side chain
        sc_ch2oh_id = olog_db.add_instance("a side chain", "CH2OH")# Serine side chain
        print(f"Added side chains with IDs: H={sc_h_id}, CH3={sc_ch3_id}, CH2OH={sc_ch2oh_id}")

        # Add some alpha carbons (conceptual, could add more detail)
        ac1_id = olog_db.add_instance("an alpha carbon", "Alpha Carbon 1")
        ac2_id = olog_db.add_instance("an alpha carbon", "Alpha Carbon 2")
        ac3_id = olog_db.add_instance("an alpha carbon", "Alpha Carbon 3")
        print(f"Added alpha carbons with IDs: {ac1_id}, {ac2_id}, {ac3_id}")

        # Add amino acids, linking to side chains and alpha carbons using their IDs
        # The keys in relationships match the MORPHISM LABELS marked as is_functional_fk
        gly_id = olog_db.add_instance("an amino acid", "Glycine",
                                      has_side_chain=sc_h_id,
                                      has_alpha_carbon=ac1_id)
        ala_id = olog_db.add_instance("an amino acid", "Alanine",
                                      has_side_chain=sc_ch3_id,
                                      has_alpha_carbon=ac2_id)
        ser_id = olog_db.add_instance("an amino acid", "Serine",
                                      has_side_chain=sc_ch2oh_id,
                                      has_alpha_carbon=ac3_id)
        print(f"Added amino acids with IDs: Glycine={gly_id}, Alanine={ala_id}, Serine={ser_id}")

        # Add an amino acid instance missing a relationship (FK will be NULL)
        unknown_aa_id = olog_db.add_instance("an amino acid", "Unknown AA")
        print(f"Added amino acid 'Unknown AA' with ID: {unknown_aa_id}")


except (ConnectionError, ValueError, sqlite3.Error) as e:
    print(f"Database operation failed: {e}")

---
## 5. Query Instances and Relationships

Retrieve data from the database using the Olog methods.

In [ ]:
# --- 5. Query Instances ---
print("\n--- Querying Instances ---")
try:
    with amino_acid_olog as olog_db:
        # Get all amino acids
        all_aas = olog_db.get_instances("an amino acid")
        print("\nAll Amino Acids:")
        for aa_row in all_aas:
            # Access columns by name (thanks to row_factory) or index
            print(f"  ID: {aa_row['id']}, Value: {aa_row['value']}, "
                  f"SideChainFK: {aa_row['has_side_chain_id']}, AlphaCarbonFK: {aa_row['has_alpha_carbon_id']}")

        # Get a specific amino acid (e.g., Alanine, assuming its ID is ala_id)
        if 'ala_id' in locals() and ala_id is not None:
            alanine = olog_db.get_instance_by_id("an amino acid", ala_id)
            if alanine:
                print(f"\nAlanine (ID: {ala_id}): {dict(alanine)}") # Convert Row to dict for nice printing

                # Follow the 'has_side_chain' relationship
                print("  Following 'has_side_chain' relationship...")
                related_sc = olog_db.get_related_instance(
                    source_obj_label="an amino acid",
                    source_instance_id=ala_id,
                    morphism_label="has_side_chain"
                )
                if related_sc:
                    print(f"    -> Related Side Chain (ID: {related_sc['id']}): {related_sc['value']}")
                else:
                    print(f"    -> No related side chain found or FK was NULL.")

                # Follow the 'has_alpha_carbon' relationship
                print("  Following 'has_alpha_carbon' relationship...")
                related_ac = olog_db.get_related_instance(
                    source_obj_label="an amino acid",
                    source_instance_id=ala_id,
                    morphism_label="has_alpha_carbon"
                )
                if related_ac:
                     print(f"    -> Related Alpha Carbon (ID: {related_ac['id']}): {related_ac['value']}")
                else:
                    print(f"    -> No related alpha carbon found or FK was NULL.")

        else:
            print("\nVariable 'ala_id' not found (likely due to error during insertion). Skipping detailed query.")

        # Example with potentially NULL FK
        if 'unknown_aa_id' in locals() and unknown_aa_id is not None:
             unknown_aa = olog_db.get_instance_by_id("an amino acid", unknown_aa_id)
             if unknown_aa:
                  print(f"\nUnknown AA (ID: {unknown_aa_id}): {dict(unknown_aa)}")
                  print("  Following 'has_side_chain' relationship...")
                  related_sc_unknown = olog_db.get_related_instance("an amino acid", unknown_aa_id, "has_side_chain")
                  if related_sc_unknown:
                      print(f"    -> Related Side Chain: {dict(related_sc_unknown)}")
                  else:
                      print("    -> No related side chain found (FK is likely NULL).")


except (ConnectionError, ValueError, sqlite3.Error) as e:
    print(f"Database query failed: {e}")


print("\n--- Database Interaction Example Complete ---")
# The context manager (__exit__) ensures the connection is closed here.